In [1]:
import pdb
import numpy as np
import gcp.bigquery as bq
import gcp.storage as storage
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import SGD
import tensorflow as tf
import h5py
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation
import time
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt
from keras.layers import Dropout

HDF_FILENAME = 'final_model1.hdf5'
JSON_MODEL_FILENAME = 'final_model.json'

GRID_HDF_FILENAME = 'best_grid_model.hdf5'

seed = 13
np.random.seed(seed)

Using TensorFlow backend.


# 1. Load Data

In [2]:
# Put all categorical data first for easier implementation of One Hot Encoding.
fields_str = """
gap	day_in_week	weather_1_slots_ago	weather_2_slots_ago	weather_3_slots_ago	busy_time	
tj_level1_1_slots_ago	tj_level2_1_slots_ago	tj_level3_1_slots_ago	tj_level4_1_slots_ago	
tj_level1_2_slots_ago	tj_level2_2_slots_ago	tj_level3_2_slots_ago	tj_level4_2_slots_ago	
tj_level1_3_slots_ago	tj_level2_3_slots_ago	tj_level3_3_slots_ago	tj_level4_3_slots_ago	
temperature_1_slots_ago	pm25_1_slots_ago	
temperature_2_slots_ago	pm25_2_slots_ago	
temperature_3_slots_ago	pm25_3_slots_ago	
gap_1_slots_ago	sum_price_1_slots_ago	
gap_2_slots_ago	sum_price_2_slots_ago	
gap_3_slots_ago	sum_price_3_slots_ago	
f1	f11	f11_1	f11_2	f11_3	f11_4	f11_5	f11_6	f11_7	
f11_8	f13_4	f13_8	f14	f14_1	f14_10	f14_2	f14_3	f14_6	f14_8	f15	f15_1	
f15_2	f15_3	f15_4	f15_6	f15_7	f15_8	f16	f16_1	f16_10	f16_11	f16_12	f16_3	
f16_4	f16_6	f17	f17_2	f17_3	f17_4	f17_5	f19	f19_1	f19_2	f19_3	f19_4	f1_1	
f1_10	f1_11	f1_2	f1_3	f1_4	f1_5	f1_6	f1_7	f1_8	f20	f20_1	f20_2	
f20_4	f20_5	f20_6	f20_7	f20_8	f20_9	f21_1	f21_2	f22	f22_1	f22_2	f22_3	
f22_4	f22_5	f23	f23_1	f23_2	f23_3	f23_4	f23_5	f23_6	f24	f24_1	f24_2	f24_3	
f25	f25_1	f25_3	f25_7	f25_8	f25_9	f2_1	f2_10	f2_11	f2_12	f2_13	f2_2	
f2_4	f2_5	f2_6	f2_7	f2_8	f3_1	f3_2	f3_3	f4	f4_1	f4_10	f4_11	
f4_13	f4_14	f4_16	f4_17	f4_18	f4_2	f4_3	f4_5	f4_6	f4_7	f4_8	f4_9	
f5	f5_1	f5_3	f5_4	f6	f6_1	f6_2	f6_4	f7	f8	f8_1	f8_2	f8_3	f8_4	
f8_5
"""
fields = map(lambda x: x.strip(), fields_str.split('\t'))
features = fields[1:]

# Use this instead of len(features) since this variable can change
# e.g. when one hot encoding is used and/or new features are added.
n_features = len(features)

In [3]:
%%sql --module q_all

SELECT *, HASH(CAST(district_id AS STRING) +timeslot) AS hash_value,
  IF(ABS(HASH(CAST(district_id AS STRING) + timeslot)) % 2 == 1, 'True', 'False')
    AS included_in_sample, IF(timeofday_slot >= 50 AND timeofday_slot <= 53, 1, 0) AS busy_time
FROM [datalab-projects-1331:xjk_algo_comp.gaps]
WHERE gap > 0
LIMIT 10000

# The above query randomizes its outputs.

In [4]:
query = bq.Query(q_all)
tableresult = query.results()

all_data = np.zeros((tableresult.length, len(fields)))
print 'there are {} rows'.format(tableresult.length)
for rcounter, row in enumerate(tableresult):
  for fcounter, field in enumerate(fields):
    all_data[rcounter, fcounter] = row[field]
  if rcounter % 5000 == 0:
    print 'processed {} rows'.format(rcounter)
all_data_original = np.copy(all_data)

there are 10000 rows
processed 0 rows
processed 5000 rows


# 2. Preprocess Data

Note that in the end we don't use data adjusted from Preprocessing steps, but rather including the preprocessing instances in Pipeline to be applied to original data.

In [5]:
all_data = np.copy(all_data_original)

## 2.1. Impute NaN and inf values

In [6]:
from sklearn.preprocessing import Imputer
# Useful code to check NaN and Inf values. This is needed since these values would
# cause "Input contains NaN, infinity or a value too large for dtype('float32')
# errors when left unchecked.
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

# Impute
imputer = Imputer()
imputer.fit(all_data)
print 'Shape of old data:'
print all_data.shape
print 'Stats (NaN values are replaced with following means for each feature):'
print imputer.statistics_
all_data = imputer.fit_transform(all_data)
print 'Shape of new data:'
print all_data.shape

# See that NaN and Inf values replaced
print "Checkinf for NaN and Inf"
print "np.nan=", np.where(np.isnan(all_data))
print "is.inf=", np.where(np.isinf(all_data))
print "np.max=", np.max(abs(all_data))

Checkinf for NaN and Inf
np.nan= (array([  25,   25,   32, ..., 9991, 9991, 9991]), array([24, 25,  6, ..., 27, 28, 29]))
is.inf= (array([], dtype=int64), array([], dtype=int64))
np.max= nan
Shape of old data:
(10000, 164)
Stats (NaN values are replaced with following means for each feature):
[  1.34958000e+01   2.14040000e+00   4.09972568e+00   4.03141309e+00
   3.99384451e+00   3.06000000e-02   9.58967328e+02   2.07643385e+02
   6.16082339e+01   4.20156282e+01   9.59768764e+02   2.08709724e+02
   6.19740374e+01   4.21995414e+01   9.60672129e+02   2.09511531e+02
   6.21194619e+01   4.23145863e+01   6.84873879e+00   1.43985565e+02
   6.86372608e+00   1.43527834e+02   6.88533721e+00   1.43774137e+02
   1.37731013e+01   1.32076869e+03   1.37133305e+01   1.33305655e+03
   1.39385155e+01   1.34528138e+03   1.72666560e+03   7.48571190e+03
   1.35009460e+03   3.32622500e+03   3.81518630e+03   2.24369252e+04
   1.11232450e+03   8.35454760e+03   1.68802080e+03   2.07812412e+04
   1.93056257e+0

## 2.2. One Hot Encoding

In [7]:
one_hot = OneHotEncoder(categorical_features=[1, 2, 3, 4], n_values=[7, 10, 10, 10])
one_hot.fit(all_data)
print "n_values_:"
print one_hot.n_values_
print "feature_indices_:"
print one_hot.feature_indices_
all_data = one_hot.fit_transform(all_data).todense()
n_features = all_data.shape[1] - 1
print 'new number of features: {}'.format(n_features)

n_values_:
[ 7 10 10 10]
feature_indices_:
[ 0  7 17 27 37]
new number of features: 196


## 2.3. Feature Scaling

In [8]:
scaler = StandardScaler()
print 'old data:'
print 'means: {}, min: {}, max: {}'.format(np.mean(all_data), np.min(all_data), np.max(all_data))
all_data = scaler.fit_transform(all_data)
print 'new data:'
print 'means: {}, min: {}, max: {}'.format(np.mean(all_data), np.min(all_data), np.max(all_data))

old data:
means: 2571.49518, min: 0.0, max: 461563.0
new data:
means: 4.72060067291e-17, min: -1.93858841442, max: 23.8210577908


# 3. Split into Train and Test Data

In [9]:
# Split the originaldata into train and test sets.
data_size = all_data_original.shape[0]
training_size = data_size * 90/100
indices = np.random.permutation(data_size)
training_idx, test_idx = indices[:training_size], indices[training_size:]
all_data_train, all_data_test = all_data_original[training_idx,:], all_data_original[test_idx,:]

data_train = all_data_train[:,1:]
targets_train = all_data_train[:,0]
data_test = all_data_test[:,1:]
targets_test = all_data_test[:,0]

# 4. Model Creation

## 4.1. Loss Function

In [10]:
def mape(y, predictions):
  return K.mean(K.abs(y-predictions/K.clip(K.abs(y), K.epsilon(), np.inf)), axis=-1)

## 4.2. Create Pipeline

In [11]:
def create_model(optimizer='adagrad', init='uniform'):
  model = Sequential()
  model.add(Dense(n_features, input_dim=n_features, init=init, activation='relu'))
  model.add(Dense(int(n_features/2), init=init, activation='relu'))
  model.add(Dense(int(n_features/4), init=init, activation='relu'))
  model.add(Dense(1))
  model.compile(loss=mape, optimizer=optimizer)
  return model

model = KerasRegressor(build_fn=create_model, nb_epoch=20, batch_size=10, verbose=2)

steps = [
  ('impute', Imputer()),
  # Remember that gap, the first variable, is removed, thus categorical_features
  # should start at index 0.
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False)),
  ('scale', StandardScaler()),
#   ('pca', PCA(n_components=120)),
  ('estimate', model)
]
final_est = Pipeline(steps)

# kfold = StratifiedKFold(y=targets_train, n_folds=10, shuffle=True, random_state=seed)

## 4.3. Grid Search to Find Best Hyperparameters

Use subset of training data in grid search just to find what hyperparameters are best for this project. Specifically we want to find the best optimizer, initializer, and batch size. Topology of the model will be adjusted in crossvalidation step.

In [12]:
# model.fit(data_train, targets_train, validation_split=0.33,  nb_epoch=100, batch_size=10, 
#           callbacks=callbacks_list)
# results = cross_val_score(model, data_train, targets_train, cv=kfold)

# checkpoint = ModelCheckpoint(GRID_HDF_FILENAME, monitor='loss', save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# epochs = 150
# learning_rate = 0.1
# decay_rate = learning_rate / epochs
# momentum = 0.8
# sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

# data_train_s = data_train[:1000]
# targets_train_s = targets_train[:1000]
# optimizers = [sgd, 'adam', 'adagrad']
# init = ['glorot_uniform', 'normal', 'uniform']
# epochs_r = np.array([epochs])
# batches = np.array([5, 10, 20])
# param_grid = {
#   'estimate__optimizer': optimizers,
#   'estimate__nb_epoch': epochs_r, 
#   'estimate__batch_size': batches,
#   'estimate__init': init,
#   'estimate__callbacks': [callbacks_list]
# }

# grid = GridSearchCV(estimator=final_est, param_grid=param_grid, cv=3)
# grid_result = grid.fit(data_train_s, targets_train_s)
# time.sleep(0.5)

In [13]:
# summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# for params, mean_score, scores in grid_result.grid_scores_:
# print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [14]:
# scores = model.evaluate(data_train, targets_train)
# print("\nscore: %.2f%" % (scores))

In [15]:
# scores = model.evaluate(data_test, targets_test)
# print("\nscore: %.2f%" % (scores))

## 4.4. Crossvalidation

Use entire training data for cross validation. Resulting model from here will be the final model.

In [16]:
steps = [
  ('impute', Imputer()),
  # Remember that gap, the first variable, is removed, thus categorical_features
  # should start at index 0.
  ('one_hot', OneHotEncoder(categorical_features=[0, 1, 2, 3], sparse=False,
                           n_values=[7, 10, 10, 10])),
  ('scale', StandardScaler()),
#   ('pca', PCA(n_components=120)),
#   ('estimate', final_model)
]
transformer = Pipeline(steps)
data_train2 = transformer.fit_transform(data_train)

In [ ]:
epochs = 1000

def get_optimizer(epochs=50):
  learning_rate = 1
  decay_rate = learning_rate / epochs
  momentum = 0.8
  return SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

def create_final_model():
  init='uniform'
  final_model = Sequential()
  final_model.add(Dropout(0.2, input_shape=(n_features,)))
  final_model.add(Dense(n_features, init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(int(n_features/2), init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(int(n_features/4), init=init, activation='relu'))
  final_model.add(Dropout(0.2))
  final_model.add(Dense(1))
  final_model.compile(loss=mape, optimizer=get_optimizer(epochs))
  return final_model

final_est = KerasRegressor(build_fn=create_final_model)

In [ ]:
checkpoint = ModelCheckpoint(HDF_FILENAME, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

history = final_est.fit(data_train2, targets_train, validation_split=0.33,
                          nb_epoch=epochs, batch_size=10, verbose=0, callbacks=callbacks_list)
time.sleep(0.1)

Epoch 00000: val_loss did not improve
Epoch 00001: val_loss did not improve
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00006: val_loss did not improve


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# kfold = KFold(n=len(data_train), n_folds=10, random_state=seed)
# results = cross_val_score(estimator, data_train, targets_train, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

# Store Model to File

In [ ]:
model_json = final_est.model.to_json()
with open(JSON_MODEL_FILENAME, "w") as json_file:
  json_file.write(model_json)

In [ ]:
from keras.models import model_from_json
json_file = open(JSON_MODEL_FILENAME, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(HDF_FILENAME)
print("Loaded model from disk")

# evaluate loaded model on test data
data_test2 = transformer.fit_transform(data_test)
loaded_model.compile(loss=mape, optimizer=get_optimizer(epochs))
score = loaded_model.evaluate(data_test2, targets_test, verbose=0)
print "%s: %.2f" % (loaded_model.metrics_names[0], score)
predictions = loaded_model.predict(data_test2)
print 'sample predictions:'
print(predictions[0:10])